# Todoist Completed Tasks Downloader

This project will collect and aggregate all of your completed task data from Todoist. 

-------

## Installation and Setup

#### Download and Install Todoist Python Library

`$ pip install python-todoist`

#### Signup and Create a Todoist App

* Signup at https://developer.todoist.com/appconsole.html
* Once app is created, generate and copy a "Test token," which provides access to API as your user.
* Copy sample-credentials.json and create credentials.json
* Add and Save your Test Token to credentials.json


-----

## Dependencies

In [55]:
from todoist.api import TodoistAPI
import pandas as pd

import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline

### Credentials and Authentification

In [2]:
import json

with open("credentials.json", "r") as file:
    credentials = json.load(file)
    todoist_cr = credentials['todoist']
    TOKEN = todoist_cr['TOKEN']

In [47]:
api = TodoistAPI(TOKEN)
api.sync()

----------

## Check Basic User Info

In [4]:
user = api.state['user']

In [5]:
user['full_name']

'Mark Koester'

In [6]:
# Tasks Completed Today
user['completed_today']

6

In [7]:
# total completed tasks
user_completed_count = user['completed_count']
user_completed_count

3867

-------

# List and Export of Current Projects

In [8]:
user_projects  = api.state['projects']

In [ ]:
user_projects

In [9]:
len(user_projects)

41

In [10]:
with open('data/todoist-projects.csv', 'w') as file:
    file.write("Id" + "," + "Project" + "\n")
    for i in range(0, len(user_projects)):
        file.write('\"' + str(user_projects[i]['id']) + '\"' + "," + '\"' + str(user_projects[i]['name']) + '\"' + "\n")

In [11]:
projects = pd.read_csv("data/todoist-projects.csv")

In [12]:
# projects

-----

## User Stats Info

In [13]:
stats = api.completed.get_stats()

In [14]:
# total completed tasks from stats
stats['completed_count']

3867

-------

# Collect Raw List of All Completed Items from Todoist

In [42]:
def get_completed_todoist_items():
    # create df from initial 50 completed tasks
    print("Collecting Initial 50 Completed Todoist Tasks...")
    temp_tasks_dict = (api.completed.get_all(limit=50))
    tasks = pd.DataFrame.from_dict(temp_tasks_dict['items'])
    # get the remaining items
    pager = list(range(50,user_completed_count,50))
    for count, item in enumerate(pager):
        tmp_tasks = (api.completed.get_all(limit=50, offset=item))
        tmp_tasks_df = pd.DataFrame.from_dict(tmp_tasks['items'])
        tasks = pd.concat([tasks, tmp_tasks_df])
        print("Collecting Additional Todoist Tasks " + str(item) + " of " + str(user_completed_count))
    # save to CSV
    print("...Generating CSV Export")
    tasks.to_csv("data/todost-raw-tasks-completed.csv", index=False)

In [16]:
get_completed_todoist_items()

...Generating CSV Export


In [17]:
tasks = pd.read_csv("data/todost-raw-tasks-completed.csv")

In [18]:
tasks.head()

,completed_date,content,id,meta_data,project_id,task_id,user_id
0,Wed 23 May 2018 11:26:24 +0000,Initial Code for Exporting Todoist Completed T...,2662789071,NaN,2165379308,2662789071,4288657
1,Wed 23 May 2018 11:26:24 +0000,Edit and Share Last.fm Code for QS Ledger,2662788785,NaN,2165379308,2662788785,4288657
2,Wed 23 May 2018 11:26:21 +0000,Edit and Share Fitbit Code for QS Ledger,2662788601,NaN,2165379308,2662788601,4288657
3,Wed 23 May 2018 11:26:21 +0000,QS Ledger Dev Kickoff,2662788072,NaN,2165379308,2662788072,4288657
4,Wed 23 May 2018 05:41:36 +0000,DRAFT: Tracking Desktop with Auto-Capture Scre...,2662395597,NaN,1252539618,2662395597,4288657


In [19]:
# generated count 
collected_total = len(tasks)
collected_total

3867

In [20]:
# Does our collected total tasks match stat of completed count on user
collected_total == user_completed_count

True

In [21]:
tasks['project_id'] = tasks.project_id.astype('category')

In [22]:
tasks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3867 entries, 0 to 3866
Data columns (total 7 columns):
completed_date    3867 non-null object
content           3867 non-null object
id                3867 non-null int64
meta_data         0 non-null float64
project_id        3867 non-null category
task_id           3867 non-null int64
user_id           3867 non-null int64
dtypes: category(1), float64(1), int64(3), object(2)
memory usage: 188.1+ KB


In [23]:
len(tasks.project_id.unique())

59

---------

## Get All Current and Previous Projects

In [24]:
# get all project ids used on tasks
project_ids = tasks.project_id.unique()
project_ids

[2165379308, 1252539618, 1252630449, 1255395900, 1255273555, ..., 189007639, 187287545, 142200794, 181321548, 178797634]
Length: 59
Categories (59, int64): [2165379308, 1252539618, 1252630449, 1255395900, ..., 187287545, 142200794, 181321548, 178797634]

In [25]:
# total all-time projects
len(project_ids)

59

In [26]:
def get_todoist_project_name(project_id):
    item = api.projects.get_by_id(project_id)
    if item: 
        try:
            return item['name']
        except:
            return item['project']['name']

In [27]:
get_todoist_project_name(183682060)

'Math'

In [28]:
get_todoist_project_name(1252539618)

'Writing'

In [29]:
project_names = []
for i in project_ids:
    project_names.append(get_todoist_project_name(i))

In [49]:
# project_names

-----

## Match Project Id Name on Tasks 

In [32]:
tasks.tail()

,completed_date,content,id,meta_data,project_id,task_id,user_id
3862,Sun 28 Aug 2016 12:07:13 +0000,Read Checklist from MASTER THE GAME,53281331,NaN,178797715,53281331,4288657
3863,Sun 28 Aug 2016 10:40:51 +0000,Weekly Review,53277061,NaN,142200795,53277061,4288657
3864,Sun 28 Aug 2016 10:24:01 +0000,Financial Reflection Writing,53275224,NaN,142200795,53275224,4288657
3865,Sun 28 Aug 2016 10:17:51 +0000,Study Todoist Shortcuts,53273289,NaN,142200795,53273289,4288657
3866,Sun 28 Aug 2016 06:39:21 +0000,Review & Setup Tasks on TODOIST,53265021,NaN,142200795,53265021,4288657


In [33]:
project_lookup = lambda x: get_todoist_project_name(x)

In [34]:
tasks['project_name'] = tasks['project_id'].apply(project_lookup)

In [35]:
tasks.tail()

,completed_date,content,id,meta_data,project_id,task_id,user_id,project_name
3862,Sun 28 Aug 2016 12:07:13 +0000,Read Checklist from MASTER THE GAME,53281331,NaN,178797715,53281331,4288657,Studies: General
3863,Sun 28 Aug 2016 10:40:51 +0000,Weekly Review,53277061,NaN,142200795,53277061,4288657,Personal
3864,Sun 28 Aug 2016 10:24:01 +0000,Financial Reflection Writing,53275224,NaN,142200795,53275224,4288657,Personal
3865,Sun 28 Aug 2016 10:17:51 +0000,Study Todoist Shortcuts,53273289,NaN,142200795,53273289,4288657,Personal
3866,Sun 28 Aug 2016 06:39:21 +0000,Review & Setup Tasks on TODOIST,53265021,NaN,142200795,53265021,4288657,Personal


In [36]:
len(tasks.project_name.unique())

41

In [37]:
len(tasks)

3867

In [38]:
# Add Day of Week
tasks['completed_date'] = pd.to_datetime(tasks['completed_date'])
tasks['dow'] = tasks['completed_date'].dt.weekday
tasks['day_of_week'] = tasks['completed_date'].dt.weekday_name

In [39]:
# save to CSV
tasks.to_csv("data/todost-tasks-completed.csv", index=False)

---

# TODO: Simple Data Analysis

In [69]:
# UNCOMMMENT TO VIEW: Report by Projects
# tasks.groupby(['project_name']).count()

In [74]:
# ax = tasks.groupby(['project_name']).count().plot(kind='bar')
#plt.suptitle('Tasks Completed of Top Projects', fontsize=16)
#plt.xlabel('Projects', fontsize=12, color='red')